In [1]:
import pandas as pd
import numpy as np

from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from gensim.models import LdaModel

In [2]:
df = pd.read_csv('../ready.csv').dropna()

In [3]:
# Есть ли пропуски:
print(df.isnull().sum())
# Есть ли дубликаты
print(df.duplicated().any())
print(len(df))

Unnamed: 0.2                       0
Unnamed: 0.1                       0
Unnamed: 0                         0
ucid                               0
text_employer                      0
ACTION_ITEM_RESULT_PRODUCT_NAME    0
dtype: int64
False
42620


In [4]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ucid,text_employer,ACTION_ITEM_RESULT_PRODUCT_NAME
0,0,0,0,"5,01000641613474E+017",добрый день это клиентский менеджер виктория с...,Бизнес-карта
1,1,1,1,"5,01001121614156E+017",добрый день звать клиентский менеджер сбербанк...,Зарплатные проекты
2,2,2,2,"5,01001811614178E+017",добрый день звать сбербанк блок сбер бизнес об...,Зарплатные проекты
3,3,3,3,"5,01002531614256E+017",добрый день клиентский менеджер удобно разгова...,Бизнес-карта
4,4,4,4,"5,01005041613457E+017",добрый день это сбербанк блок сбер бизнес клие...,Зарплатные проекты


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df.text_employer, test_size=0.3, random_state=42)

In [7]:
from gensim import corpora

# Словарь
texts = [[word for word in doc.split()] for doc in X_train]
dictionary = corpora.Dictionary(texts)

# Корпус на основе матрицы подсчета слов
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
# Построение LDA модели
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=16,
                     random_state=42,
                     update_every=1,
                     chunksize=500,
                     passes=15,
                     alpha='auto',
                     per_word_topics=True)

In [9]:
lda_model.save('../models/lda.model')

In [10]:
# Сложность модели
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

texts = [[word for word in doc.split()] for doc in X_test]
# Когеренция
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.311725654424939

Coherence Score:  0.5114780003801879


In [3]:
# Вывод тем и соответствующих им ключевых слов
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}')

Topic: 0 
Words: 0.035*"пожалуйста" + 0.032*"линия" + 0.031*"информация" + 0.028*"мочь" + 0.021*"вопрос" + 0.017*"обращаться" + 0.017*"бизнес" + 0.017*"организация" + 0.016*"ваш" + 0.016*"уточнить"
Topic: 1 
Words: 0.147*"зарплатный" + 0.133*"проект" + 0.052*"выплата" + 0.031*"мочь" + 0.023*"ваш" + 0.020*"реестр" + 0.019*"перечисление" + 0.018*"договор" + 0.017*"подключить" + 0.017*"помощь"
Topic: 2 
Words: 0.071*"почта" + 0.061*"номер" + 0.047*"направить" + 0.043*"телефон" + 0.042*"электронный" + 0.042*"мочь" + 0.041*"указать" + 0.037*"информация" + 0.017*"точка" + 0.016*"инструкция"
Topic: 3 
Words: 0.023*"код" + 0.017*"нужно" + 0.017*"это" + 0.015*"зайти" + 0.015*"заявление" + 0.015*"нажимать" + 0.014*"бизнес" + 0.012*"компьютер" + 0.012*"далее" + 0.012*"заходить"
Topic: 4 
Words: 0.026*"мочь" + 0.026*"ваш" + 0.022*"контакт" + 0.022*"нный" + 0.022*"удалой" + 0.021*"свой" + 0.018*"вопрос" + 0.017*"бизнес" + 0.017*"направить" + 0.016*"сервис"
Topic: 5 
Words: 0.044*"это" + 0.029*"мочь

In [12]:
import pyLDAvis

# Визуализация топиков
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.163666  0.085720       1        1  16.849181
10     0.126602  0.178388       2        1  13.560462
6      0.115367 -0.044837       3        1  12.131537
11    -0.254444  0.198128       4        1  11.941802
13     0.174205  0.045100       5        1   6.684702
0     -0.113390 -0.092864       6        1   5.787290
3      0.031107 -0.209680       7        1   5.602127
5      0.067271 -0.064684       8        1   5.362322
7      0.210336 -0.013089       9        1   5.010420
9      0.068919 -0.084173      10        1   3.445842
12     0.088667  0.080686      11        1   3.303219
15    -0.222463 -0.004796      12        1   2.844107
4     -0.139509  0.072412      13        1   2.120121
2     -0.143729 -0.259970      14        1   1.842256
1      0.057821 -0.040255      15        1   1.813646
8      0.096907  0.153913      16        1   1.700967, topic_info=           Term          Freq         Total Category  logprob  loglift
46        карта  99891.000000  99891.000000  Default  30.0000  30.0000
0        бизнес  70578.000000  70578.000000  Default  29.0000  29.0000
256  зарплатный  17790.000000  17790.000000  Default  28.0000  28.0000
110        счёт  34122.000000  34122.000000  Default  27.0000  27.0000
6        добрый  44059.000000  44059.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
313    работать    710.200000   6740.030607  Topic16  -4.4398   1.8237
534     кабинет    784.400997  10689.689745  Topic16  -4.3404   1.4619
9          мочь   1117.297778  61752.040848  Topic16  -3.9867   0.0618
268      личный    800.590797  14330.630963  Topic16  -4.3200   1.1892
536        либо    626.558326  14157.345343  Topic16  -4.5651   0.9562

[1143 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1704      5  0.969749    автономный
2637     15  0.982741     агентство
1904      4  0.949941  адаптировать
341       3  0.022973         адрес
341       7  0.398930         адрес
...     ...       ...           ...
444       8  0.991945    юридически
1071     11  0.997009         юрист
2196      1  0.997476        январь
2004      7  0.260679        яндекс
2004     14  0.736853        яндекс

[2943 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 11, 7, 12, 14, 1, 4, 6, 8, 10, 13, 16, 5, 3, 2, 9])

In [14]:
# Функция для получения топика из текста
def get_topic(text):
    bow_vector = dictionary.doc2bow(text.split())
    topics_distribution = lda_model.get_document_topics(bow_vector)

    # Возвращаем номер топика с наибольшей вероятностью для данного документа
    return max(topics_distribution, key=lambda item: item[1])[0]

# Добавление столбца с номером топика для каждой строки
df['topics'] = df['text_employer'].apply(get_topic)

In [16]:
df[['ucid', 'text_employer', 'topics']]

,ucid,text_employer,g_topics
0,"5,01000641613474E+017",добрый день это клиентский менеджер виктория с...,10
1,"5,01001121614156E+017",добрый день звать клиентский менеджер сбербанк...,14
2,"5,01001811614178E+017",добрый день звать сбербанк блок сбер бизнес об...,11
3,"5,01002531614256E+017",добрый день клиентский менеджер удобно разгова...,11
4,"5,01005041613457E+017",добрый день это сбербанк блок сбер бизнес клие...,13
...,...,...,...
42721,"5,01034111613466E+017",добрый день это ваш персональный менеджер сбер...,2
42722,"5,01034111613466E+017",здравствуйте мочь обращаться линия операционны...,0
42723,"5,01035251614262E+017",добрый день персональный менеджер сбербанк ваш...,14
42724,"5,01035571614254E+017",добрый день персональный менеджер сбер бизнес ...,11
